In [14]:
import os
import sys
from pathlib import Path
sys.path.append('..') 
from PyPDF2 import PdfMerger
from utils.behavior.session_utils import load_session_df, parse_session_string
from utils.basics.data_org import *
import fitz

In [ ]:
from PyPDF2 import PdfMerger, PdfReader
from PyPDF2.pdf import PageObject
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import io

def add_title_to_page(page, title):
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    can.drawString(30, 750, title)  # Adjust coordinates as needed
    can.save()
    packet.seek(0)
    title_page = PdfReader(packet).pages[0]
    page.merge_page(title_page)
    return page

In [25]:
def merge_sesssion_pdf(session_list, output, prefix = '', postfix = '', strings = [''], add_title = False):
    merger = PdfMerger()
    modified_pdfs = []
    temp_folder = os.path.join(os.path.dirname(output), 'pdf_tmp')
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder, exist_ok=True)
    for session in session_list:
        session_dir = parse_session_string(session)
        curr_files = [os.path.join(session_dir['saveFigFolder'], file) for file in os.listdir(session_dir['saveFigFolder']) if file.endswith(postfix) and file.startswith(prefix) and all(s in file for s in strings)]
        if len(curr_files)>1:
            strings.append(session)
            curr_files = [os.path.join(session_dir['saveFigFolder'], file) for file in os.listdir(session_dir['saveFigFolder']) if file.endswith(postfix) and file.startswith(prefix) and all(s in file for s in strings)]
        
        if len(curr_files)!=1:
            print(f'{session} {len(curr_files)}')

        for file_ind, curr_file in enumerate(curr_files):
            if add_title:
                doc = fitz.open(curr_file)
                first_page = doc[0]
                first_page.insert_text((50, 50), session, fontsize=16, color=(0, 0, 0), fontname="helv")
                modified_pdf = os.path.join(os.path.dirname(curr_file), 'pdf_tmp', f'{session}_modified_{file_ind}.pdf')
                doc.save(modified_pdf) 
                modified_pdfs.append(modified_pdf) 
                merger.append(modified_pdf)        
            else:
                merger.append(curr_file)
    shutil.rmtree(temp_folder)
    merger.write(output)
    


In [10]:
session_list = get_session_list('photometry', 'all', 'crusIgood') + get_session_list('photometry_Gi', 'all', 'Gi_G8')

In [11]:
# preprocessing
merge_sesssion_pdf(session_list, r'F:\FP_tri-exp_prep.pdf', postfix = '_tri-exp_FP.pdf')

In [12]:
merge_sesssion_pdf(session_list, r'F:\FP_exp_prep.pdf', postfix = '_exp_FP.pdf')

In [13]:
merge_sesssion_pdf(session_list, r'F:\FP_bright_prep.pdf', postfix = '_bright_FP.pdf')

In [24]:
# compare G and Iso raw trace
merge_sesssion_pdf(session_list, r'F:\G_vs_Iso_raw_trace.pdf', postfix = '_G_vs_Iso.pdf', add_title=True)

Exception: need font file or buffer